In [1]:
"""Run two below lines to get my_abc module"""
# !git clone https://github.com/thanhttttt/thanh.git
# !pip install -r /content/thanh/requirements.txt
"""Run two below lines to drive"""
# from google.colab import drive
# drive.mount('/content/drive')

'Run two below lines to drive'

# Import

In [1]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


import sys
sys.path.append('../')
sys.path.append('/content/thanh/')
sys.path.append('../thanh/')
import sionna

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.config.seed = 42 # Set seed for reproducible random number generation

# Load the required Sionna components
from sionna.nr.my_abc import *

# Label data field

In [2]:
def update_pusch_records(pusch_records, simulator, snr_in_flexRan, filename, foldername):
    pusch_records.append(PuschRecord(
                            nPhyCellId=simulator.pusch_config.carrier.n_cell_id,
                            nSFN=simulator.pusch_config.carrier.frame_number,
                            nSlot=simulator.pusch_config.carrier.slot_number,
                            nPDU=1,
                            nGroup=1,
                            nUlsch=1,
                            nUlcch=0,
                            nRachPresent=0,
                            nRNTI=simulator.pusch_config.n_rnti,
                            nUEId=0,
                            nBWPSize=simulator.pusch_config.n_size_bwp,
                            nBWPStart=simulator.pusch_config.n_start_bwp,
                            nSubcSpacing=simulator.pusch_config.carrier.mu,
                            nCpType=simulator.pusch_config.My_Config.Sys.CpType,
                            nULType=0,
                            nMcsTable=simulator.pusch_config.tb.mcs_table - 1,
                            nMCS=simulator.pusch_config.tb.mcs_index,
                            nTransPrecode=simulator.pusch_config.My_Config.Ue[0].TransformPrecoding,
                            nTransmissionScheme=simulator.pusch_config.My_Config.Ue[0].CodeBookBased,
                            nNrOfLayers=simulator.pusch_config.num_layers,
                            nPortIndex=simulator.pusch_config.dmrs.dmrs_port_set,
                            nNid=simulator.pusch_config.tb.n_id,
                            nSCID=simulator.pusch_config.dmrs.n_scid,
                            nNIDnSCID=simulator.pusch_config.dmrs.n_id[0],
                            nNrOfAntennaPorts=simulator.pusch_config.My_Config.Sys.NRxAnt,
                            nVRBtoPRB=0,
                            nPMI=simulator.pusch_config.My_Config.Ue[0].Tpmi,
                            nStartSymbolIndex=simulator.pusch_config.symbol_allocation[0],
                            nNrOfSymbols=simulator.pusch_config.symbol_allocation[1],
                            nResourceAllocType=1,
                            nDMRSTypeAPos=simulator.pusch_config.dmrs.type_a_position,
                            nRBStart=simulator.pusch_config.first_resource_block,
                            nRBSize=simulator.pusch_config.num_resource_blocks,
                            nTBSize=(simulator.pusch_config.tb_size//8),
                            nRV=simulator.pusch_config.My_Config.Sys.rvSeq,
                            nHARQID=0,
                            nNDI=1,
                            nMappingType=simulator.pusch_config.My_Config.Ue[0].PuschMappingType,
                            nDMRSConfigType=simulator.pusch_config.My_Config.Ue[0].DmrsConfigurationType,
                            nNrOfCDMs=simulator.pusch_config.dmrs.num_cdm_groups_without_data,
                            nNrOfDMRSSymbols=simulator.pusch_config.dmrs.length,
                            nDMRSAddPos=simulator.pusch_config.dmrs.additional_position,
                            nPTRSPresent=simulator.pusch_config.My_Config.Ue[0].Ptrs,
                            nAck=simulator.pusch_config.My_Config.Ue[0].OAck,
                            nAlphaScaling=simulator.pusch_config.My_Config.Ue[0].ScalingFactor,
                            nBetaOffsetACKIndex=simulator.pusch_config.My_Config.Ue[0].IHarqAckOffset,
                            nCsiPart1=simulator.pusch_config.My_Config.Ue[0].OCsi1,
                            nBetaOffsetCsiPart1Index=simulator.pusch_config.My_Config.Ue[0].ICsi1Offset,
                            nCsiPart2=simulator.pusch_config.My_Config.Ue[0].OCsi2,
                            nBetaOffsetCsiPart2Index=simulator.pusch_config.My_Config.Ue[0].ICsi2Offset,
                            nTpPi2BPSK=simulator.pusch_config.My_Config.Ue[0].TpPi2Bpsk,
                            nTPPuschID=simulator.pusch_config.My_Config.Ue[0].NRsId,
                            nRxRUIdx=np.arange(0, simulator.pusch_config.My_Config.Sys.NRxAnt),
                            nUE=1,
                            nPduIdx=[0],
                            Channel_model=f"{'Field'}-{'bk'}",
                            Speed=0,
                            Delay_spread=0,
                            Esno_db=snr_in_flexRan,
                            Data_filename=filename,
                            Data_dirname=foldername
                        )
                    )

In [3]:
def label_field_data(model, dump_files, parquet_dir, hdf5_dir, folder_name):
    ls_counter = 0
    nn_counter = 0

    pusch_records = []
    with tqdm(total=len(dump_files), desc="Generating Data") as pbar:
        for n, filename in enumerate(dump_files):
            cfg_path = f'{filename}_cfg.txt'
            data_path = f'{filename}_freq.bin'

            caseInfo, sysInfo, ueInfo, _, auxInfo = config_parser(cfg_path)
            freqIQ = data_reader(data_path, (sysInfo['NRxAnt'], 14, sysInfo['BwpNRb']*12))
            ueIQ = tf.constant(freqIQ[None, None, :, :, ueInfo[0]['FirstPrb']*12:(ueInfo[0]['NPrb'] + ueInfo[0]['FirstPrb'])*12])

            sysCfg = SystemConfig(**sysInfo)
            ueCfg = UeConfig(**ueInfo[0])
            myCfg = MyConfig(sysCfg, [ueCfg])
            puschCfg = MyPUSCHConfig(myCfg, slot_number=caseInfo['sf'], frame_number=caseInfo['sfn'])

            snr_in_flexRan = int(auxInfo["nSNR(1)"])/10.
            simulator = MySimulator(puschCfg)
        
            r = simulator.ref(1)

            """Evaluate ls"""

            instance_name = os.path.basename(filename)
            timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
            instance_name = f'{instance_name}_{timestamp}'
            
            pbar.set_description(f'{n:3}: {instance_name:<48} | Passed: {len(pusch_records)} | nSnr in flexRan: {snr_in_flexRan:4} | MCS: {puschCfg.tb.mcs_index:2} | NRB: {puschCfg.num_resource_blocks:4}')
            pbar.update(1)  # Increment progress


            h_est, x_hat, llr_det, b_hat, crc_1 = simulator.rec(ueIQ)
            if crc_1:
                b = b_hat
                c = simulator.TB_Encoder(b)
                y = ueIQ

                assert b.shape[0] == b.shape[1] == c.shape[0] == c.shape[1] == y.shape[0] == y.shape[1] == 1

                b = tf.cast(b, dtype=tf.uint8)[0][0]
                c = tf.cast(c, dtype=tf.uint8)[0][0]
                r = tf.transpose(tf.constant(simulator.pusch_config.dmrs_grid, dtype=y.dtype), [0, 2, 1])
                y =  y[0][0]

                save_hdf5([b,c,y,r], f'{hdf5_dir}/{folder_name}', instance_name)
                update_pusch_records(pusch_records, simulator, snr_in_flexRan, instance_name, folder_name)
                ls_counter += 1
                
                continue
            """Evaluate nn"""
            preds = predict(model, ueIQ, r[0:1])
            c_soft = tf.reshape(preds, [preds.shape[0], 1, -1])
            b_hat, crc_2 = simulator.TB_Decode(c_soft) 

            if crc_2:
                b = b_hat
                c = simulator.TB_Encoder(b)
                y = ueIQ

                assert b.shape[0] == b.shape[1] == c.shape[0] == c.shape[1] == y.shape[0] == y.shape[1] == 1

                b = tf.cast(b, dtype=tf.uint8)[0][0]
                c = tf.cast(c, dtype=tf.uint8)[0][0]
                r = tf.transpose(tf.constant(simulator.pusch_config.dmrs_grid, dtype=y.dtype), [0, 2, 1])
                y =  y[0][0]

                
                save_hdf5([b,c,y,r], f'{hdf5_dir}/{folder_name}', instance_name)
                update_pusch_records(pusch_records, simulator, snr_in_flexRan, instance_name, folder_name)
                nn_counter += 1


            


            if (n+1) % 16 == 0:
                df = pd.DataFrame.from_records(pusch_records, columns=PuschRecord._fields)
                df.to_parquet(f'{parquet_dir}/{folder_name}.parquet', engine="pyarrow")

        df = pd.DataFrame.from_records(pusch_records, columns=PuschRecord._fields)
        df.to_parquet(f'{parquet_dir}/{folder_name}.parquet', engine="pyarrow")
        


In [4]:
# Đường dẫn đến thư mục chứa các file config
config_dir = "/workspaces/thanh/data_field/"

filtered_dump_fail_files = []
filtered_dump_pass_files = []

# Duyệt qua các file trong thư mục
for root, dirs, files in os.walk(config_dir):
    for name in files:
        filename = f'{root}/{name}'
        if filename.endswith("_cfg.txt"):  # Chỉ xét các file config
            file_path = os.path.join(config_dir, filename)
            
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()

            DmrsTypeAPos = re.search(r"ue\{1\}\.DmrsTypeAPosition\s*=\s*(\d+)", content)
            DmrsAddPos = re.search(r"ue\{1\}\.DmrsAdditionalPosition\s*=\s*(\d+)", content)
            Mcs = re.search(r"ue\{1\}\.Mcs\s*=\s*(\d+)", content)
            Nrb = re.search(r"ue\{1\}\.NPrb\s*=\s*(\d+)", content)

            if DmrsTypeAPos and DmrsAddPos and Mcs and Nrb:
                DmrsTypeAPos = int(DmrsTypeAPos.group(1))
                DmrsAddPos = int(DmrsAddPos.group(1))
                Mcs = int(Mcs.group(1))
                Nrb = int(Nrb.group(1))

                # Kiểm tra điều kiện
                if DmrsTypeAPos == 3 and DmrsAddPos == 1 and Mcs in {3, 5, 6, 7, 8, 9} and Nrb >= 4:
                    filename = filename[:-len('_cfg.txt')]
                    if 'dump_fail' in filename:
                        filtered_dump_fail_files.append(filename)
                    elif 'dump_pass' in filename:
                        filtered_dump_pass_files.append(filename)

print("Các file thỏa mãn điều kiện:")
print('dump fail: ', len(filtered_dump_fail_files), 'files', '\ndump pass:', len(filtered_dump_pass_files), 'files')

Các file thỏa mãn điều kiện:
dump fail:  52 files 
dump pass: 181 files


In [5]:
_model = CustomNeuralReceiver(training = False)
inputs = tf.zeros([1,48,14,18])
_model(inputs)
load_weights(_model, '../weight_4RB_batchsize_1024_186k_sample_dynamic_config_epoch120.pkl');

Loaded pretrained weights from ../weight_4RB_batchsize_1024_186k_sample_dynamic_config_epoch120.pkl


In [6]:
parquet_dir = '../Pusch_data/dataset/parquet'
hdf5_dir = '../Pusch_data/dataset/hdf5'
folder_name = 'field_bk_on_dump_fail_18_3_2025'
# label_field_data(_model, filtered_dump_fail_files, parquet_dir, hdf5_dir, folder_name)

# Load labeled data field as 4RB dataset

In [2]:
dataset_dir = f'../Pusch_data/dataset'
pickles_dir = f'{dataset_dir}/pickle'
hdf5_dir = f'{dataset_dir}/hdf5'
parquet_dir = f'{dataset_dir}/parquet'

In [41]:
parquet_name_pass = 'field_bk_on_dump_pass_18_3_2025'
df_pass = pd.read_parquet(f'{parquet_dir}/{parquet_name_pass}.parquet', engine="pyarrow")
parquet_name_fail = 'field_bk_on_dump_fail_18_3_2025'
df_fail = pd.read_parquet(f'{parquet_dir}/{parquet_name_fail}.parquet', engine="pyarrow")
df = pd.concat([df_pass, df_fail]).reset_index(drop=True).reset_index()
df

,index,nPhyCellId,nSFN,nSlot,nPDU,nGroup,nUlsch,nUlcch,nRachPresent,nRNTI,...,nTPPuschID,nRxRUIdx,nUE,nPduIdx,Channel_model,Speed,Delay_spread,Esno_db,Data_filename,Data_dirname
0,0,441,1015,4,1,1,1,0,0,20002,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Field-bk,0,0,16.2,dump_pass_sfn_1015_sf_4_20250318035448763254,field_bk_on_dump_pass_18_3_2025
1,1,441,1011,5,1,1,1,0,0,20002,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Field-bk,0,0,14.8,dump_pass_sfn_1011_sf_5_20250318035453113429,field_bk_on_dump_pass_18_3_2025
2,2,442,951,4,1,1,1,0,0,30004,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Field-bk,0,0,-5.2,dump_pass_sfn_951_sf_4_20250318035517805371,field_bk_on_dump_pass_18_3_2025
3,3,443,494,15,1,1,1,0,0,40002,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Field-bk,0,0,1.0,dump_pass_sfn_494_sf_15_20250318035525235492,field_bk_on_dump_pass_18_3_2025
4,4,441,1011,4,1,1,1,0,0,20002,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Field-bk,0,0,15.6,dump_pass_sfn_1011_sf_4_20250318035608214703,field_bk_on_dump_pass_18_3_2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,136,442,516,5,1,1,1,0,0,30025,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Field-bk,0,0,13.7,dump_fail_sfn_516_sf_5_20250318044107549188,field_bk_on_dump_fail_18_3_2025
137,137,443,495,14,1,1,1,0,0,40002,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Field-bk,0,0,-4.7,dump_fail_sfn_495_sf_14_20250318044117904618,field_bk_on_dump_fail_18_3_2025
138,138,441,39,15,1,1,1,0,0,20002,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Field-bk,0,0,4.7,dump_fail_sfn_39_sf_15_20250318044325085173,field_bk_on_dump_fail_18_3_2025
139,139,443,684,5,1,1,1,0,0,40002,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],Field-bk,0,0,5.3,dump_fail_sfn_684_sf_5_20250318044349621894,field_bk_on_dump_fail_18_3_2025


In [50]:
def postprocessing(index, esno_db, c, y, r):
    """
    Splits the input tensor y into 4RB chunks (48 subcarriers along axis 1) as a post-processing step.
    If the number of subcarriers is not a multiple of 48, the last incomplete chunk is kept.
    """
    # Get the number of subcarriers
    num_subcarriers = tf.shape(y)[1]
    num_chunks = num_subcarriers // 48

    c = tf.reshape(c, [12, num_subcarriers, 2])
    c = tf.reshape(c[:, :num_chunks * 48, :], [12, -1, 48, 2])
    c = tf.transpose(c, [1, 0, 2, 3])
    c = tf.reshape(c, [-1, 12*48*2])

    # Reshape the tensor to split into 4RB chunks
    y = tf.reshape(y[:, :num_chunks * 48, :, :], [-1, 48, 14, 16])
    r = tf.reshape(r[:, :num_chunks * 48, :, :], [-1, 48, 14, 2])

    return tf.repeat(index, r.shape[0]), tf.repeat(esno_db, r.shape[0]), c, y, r

In [43]:
_model = CustomNeuralReceiver(training = False)
inputs = tf.zeros([1,48,14,18])
_model(inputs)
_model.summary()

#load_weights(_model, '/content/drive/MyDrive/Pusch_data/Model_weights/model_weight_FULL_RB_epoch_40.pkl')
# load_weights(_model, '../model_weight_FULL_RB_epoch_40.pkl')
load_weights(_model, '../weight_4RB_batchsize_1024_186k_sample_dynamic_config_epoch90.pkl')

Model: "custom_neural_receiver_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          multiple                  20864     
                                                                 
 residual_block_8 (Residual  multiple                  639232    
 Block)                                                          
                                                                 
 residual_block_9 (Residual  multiple                  639232    
 Block)                                                          
                                                                 
 residual_block_10 (Residua  multiple                  639232    
 lBlock)                                                         
                                                                 
 residual_block_11 (Residua  multiple                  639232    
 lBlock)                                  

In [ ]:
# Load train_data into Dataset
original_dataset = tf.data.Dataset.from_generator(lambda: data_loader(df, hdf5_dir),
                        output_types=(tf.int32, tf.float32, tf.float32, tf.complex64, tf.complex64))

original_dataset = original_dataset.cache()
original_dataset = original_dataset.prefetch(tf.data.AUTOTUNE)
original_dataset = original_dataset.map(preprocessing).batch(1)

index_list = []
esno_db_list = []
c_list = []
y_list = []
r_list = []

for n, (index, esno_db, c, y, r) in enumerate(original_dataset):
    index, esno_db, c, y, r = postprocessing(index, esno_db, c, y, r)

    index_list.append(index)
    esno_db_list.append(esno_db)
    c_list.append(c)
    y_list.append(y)
    r_list.append(r)

# Concatenate along axis 0 to form a single dataset
index_tensor = tf.concat(index_list, axis=0)
esno_db_tensor = tf.concat(esno_db_list, axis=0)
c_tensor = tf.concat(c_list, axis=0)
y_tensor = tf.concat(y_list, axis=0)
r_tensor = tf.concat(r_list, axis=0)

In [51]:
r_tensor

<tf.Tensor: shape=(4771, 48, 14, 2), dtype=float32, numpy=
array([[[[ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         ...,
         [-1., -1.],
         [ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         ...,
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         ...,
         [ 1.,  1.],
         [ 0.,  0.],
         [ 0.,  0.]],

        ...,

        [[ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         ...,
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         ...,
         [ 1., -1.],
         [ 0.,  0.],
         [ 0.,  0.]],

        [[ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         ...,
         [ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.]]],


       [[[ 0.,  0.],
         [ 0.,  0.],
         [ 0.,  0.],
         .

In [52]:
# Create a tf.data.Dataset from stacked tensors
BATCH_SIZE = 4
new_dataset = tf.data.Dataset.from_tensor_slices((index_tensor, esno_db_tensor, c_tensor, y_tensor, r_tensor))
new_dataset = new_dataset.cache()
new_dataset = new_dataset.prefetch(tf.data.AUTOTUNE)
new_dataset = new_dataset.batch(BATCH_SIZE)

In [55]:
for n, (index, esno_db, c, y, r) in enumerate(new_dataset):
    print(y.shape, r.shape, c.shape, esno_db.shape)
    y_r = tf.concat([y, r], axis = -1) # Model input

    c_pred = _model(y_r)
    c_pred = tf.concat([c_pred[...,0:3,:],c_pred[...,4:11,:], c_pred[...,12:14,:]],axis=-2)
    c_pred = tf.transpose(c_pred, perm=[0,2,1,3])
    c_pred = tf.reshape(c_pred, [-1,(c_pred.shape[1]*c_pred.shape[2]*c_pred.shape[3])])
    print(loss_cal(c_pred, c))

(4, 48, 14, 16) (4, 48, 14, 2) (4, 1152) (4,)
tf.Tensor(0.011936756, shape=(), dtype=float32)
(4, 48, 14, 16) (4, 48, 14, 2) (4, 1152) (4,)
tf.Tensor(0.019651122, shape=(), dtype=float32)
(4, 48, 14, 16) (4, 48, 14, 2) (4, 1152) (4,)
tf.Tensor(0.022581441, shape=(), dtype=float32)
(4, 48, 14, 16) (4, 48, 14, 2) (4, 1152) (4,)
tf.Tensor(0.013478709, shape=(), dtype=float32)
(4, 48, 14, 16) (4, 48, 14, 2) (4, 1152) (4,)
tf.Tensor(0.0059292037, shape=(), dtype=float32)
(4, 48, 14, 16) (4, 48, 14, 2) (4, 1152) (4,)
tf.Tensor(0.0007648382, shape=(), dtype=float32)
(4, 48, 14, 16) (4, 48, 14, 2) (4, 1152) (4,)
tf.Tensor(0.00056102715, shape=(), dtype=float32)
(4, 48, 14, 16) (4, 48, 14, 2) (4, 1152) (4,)
tf.Tensor(0.0032333918, shape=(), dtype=float32)
(4, 48, 14, 16) (4, 48, 14, 2) (4, 1152) (4,)
tf.Tensor(0.01364284, shape=(), dtype=float32)
(4, 48, 14, 16) (4, 48, 14, 2) (4, 1152) (4,)
tf.Tensor(0.0790556, shape=(), dtype=float32)
(4, 48, 14, 16) (4, 48, 14, 2) (4, 1152) (4,)
tf.Tensor(0.

KeyboardInterrupt: 